In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# BalanceComp

`BalanceComp` is a specialized implementation of [ImplicitComponent](../../core_features/working_with_components/implicit_component.ipynb) that is intended to provide a simple way to implement most implicit equations without the need to define your own residuals.

## BalanceComp Options


In [ ]:
import openmdao.api as om
om.show_options_table("openmdao.components.balance_comp.BalanceComp")

## BalanceComp Constructor

The call signature for the `BalanceComp` constructor is:

```{eval-rst}
    .. automethod:: openmdao.components.balance_comp.BalanceComp.__init__()
        :noindex:
```

## Using the BalanceComp

`BalanceComp` allows you to add one or more state variables and its associated
implicit equations.  For each ``balance`` added to the component it
solves the following equation:

$$
  \begin{align}
  \mathcal{R}_{name} =
  \frac{f_{mult}(x,...) \times f_{lhs}(x,...) - f_{rhs}(x,...)}{f_{norm}(f_{rhs}(x,...))}
  \end{align}
$$

The optional normalization function $f_{norm}(f_{rhs})$ is computed as:

$$
  \begin{align}
  f_{norm}(f_{rhs}(x,...)) =
  \begin{cases}
   \left| f_{rhs} \right|, & \text{if normalize and } \left| f_{rhs} \right| \geq 2 \\
   0.25 f_{rhs}^2 + 1,     & \text{if normalize and } \left| f_{rhs} \right| < 2 \\
   1,                      & \text{if not normalize}
  \end{cases}
  \end{align}
$$

The following inputs and outputs are associated with each implicit state.

```{eval-rst}
=========== ======= ====================================================
Name        I/O     Description
=========== ======= ====================================================
{name}      output  implicit state variable
lhs:{name}  input   left-hand side of equation to be balanced
rhs:{name}  input   right-hand side of equation to be balanced
mult:{name} input   left-hand side multiplier of equation to be balanced
=========== ======= ====================================================
```

The default value for the `rhs:{name}` input can be set to via the
`rhs_val` argument (see arguments below). If the rhs value is fixed (e.g. 0),
then the input can be left unconnected. The `lhs:{name}` input must always have
something connected to it and should be dependent upon the value of the implicit state variable.

The multiplier is optional and will default to 1.0 if not connected.

`BalanceComp` supports vectorized implicit states. Simply provide a default
value or shape when adding the balance that reflects the correct shape.

You can provide the arguments to create a balance when instantiating a `BalanceComp`
or you can use the ``add_balance`` method to create one or more state variables after
instantiation.  The constructor accepts all the same arguments as the ``add_balance``
method:

```{eval-rst}
    .. automethod:: openmdao.components.balance_comp.BalanceComp.add_balance
       :noindex:
```

Note that the `kwargs` arguments can include any of the keyword arguments normally available
when creating an output variable with the
`add_output` method of a [Component](../../../_srcdocs/packages/core/component).

## Example:  Scalar Root Finding

The following example uses a BalanceComp to implicitly solve the
equation:

$$
    2 \cdot x^2 = 4
$$

Here, our LHS is connected to a computed value for $x^2$, the multiplier is 2, and the RHS
is 4.  The expected solution is $x=\sqrt{2}$.  We initialize $x$ with a value of 1 so that
it finds the positive root.

In [ ]:
import openmdao.api as om

prob = om.Problem()

bal = om.BalanceComp()
bal.add_balance('x', use_mult=True)

exec_comp = om.ExecComp('y=x**2', x={'val': 1}, y={'val': 1})

prob.model.add_subsystem(name='exec', subsys=exec_comp)
prob.model.add_subsystem(name='balance', subsys=bal)

prob.model.connect('balance.x', 'exec.x')
prob.model.connect('exec.y', 'balance.lhs:x')

prob.model.linear_solver = om.DirectSolver(assemble_jac=True)
prob.model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False, maxiter=100, iprint=0)

prob.setup()

prob.set_val('balance.rhs:x', 4)
prob.set_val('balance.mult:x', 2.)

# A reasonable initial guess to find the positive root.
prob['balance.x'] = 1.0

prob.run_model()

print(prob.get_val('balance.x'))

In [ ]:
import numpy as np
from numpy.testing import assert_almost_equal

assert_almost_equal(prob.get_val('balance.x'), np.sqrt(2), decimal=7)

Alternatively, we could simplify the code by using the `mult_val` argument.

In [ ]:
prob = om.Problem()

bal = om.BalanceComp()
bal.add_balance('x', use_mult=True, mult_val=2.0)

exec_comp = om.ExecComp('y=x**2', x={'val': 1}, y={'val': 1})

prob.model.add_subsystem(name='exec', subsys=exec_comp)
prob.model.add_subsystem(name='balance', subsys=bal)

prob.model.connect('balance.x', 'exec.x')
prob.model.connect('exec.y', 'balance.lhs:x')

prob.model.linear_solver = om.DirectSolver(assemble_jac=True)
prob.model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False, maxiter=100, iprint=0)

prob.setup()

prob.set_val('balance.rhs:x', 4)

# A reasonable initial guess to find the positive root.
prob.set_val('balance.x', 1.0)

prob.run_model()
print(prob.get_val('balance.x'))

In [ ]:
assert_almost_equal(prob.get_val('balance.x'), np.sqrt(2), decimal=7)

## Example:  Vectorized Root Finding

The following example uses a BalanceComp to implicitly solve the equation:

$$
    b \cdot x + c  = 0
$$

for various values of $b$, and $c$.  Here, our LHS is connected to a computed value of
the linear equation.  The multiplier is one and the RHS is zero (the defaults), and thus
they need not be connected.

In [ ]:
n = 100

prob = om.Problem()

exec_comp = om.ExecComp('y=b*x+c',
                        b={'val': np.random.uniform(0.01, 100, size=n)},
                        c={'val': np.random.rand(n)},
                        x={'val': np.zeros(n)},
                        y={'val': np.ones(n)})

prob.model.add_subsystem(name='exec', subsys=exec_comp)
prob.model.add_subsystem(name='balance', subsys=om.BalanceComp('x', val=np.ones(n)))

prob.model.connect('balance.x', 'exec.x')
prob.model.connect('exec.y', 'balance.lhs:x')

prob.model.linear_solver = om.DirectSolver(assemble_jac=True)
prob.model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False, maxiter=100, iprint=0)

prob.setup()

prob.set_val('balance.x', np.random.rand(n))

prob.run_model()

b = prob.get_val('exec.b')
c = prob.get_val('exec.c')

print(prob.get_val('balance.x'))

In [ ]:
print(-c/b)  # expected

In [ ]:
assert_almost_equal(prob.get_val('balance.x'), -c/b, decimal=6)
assert_almost_equal(-c/b, prob.get_val('balance.x'), decimal=6)  # expected

## Example:  Providing an Initial Guess for a State Variable

`BalanceComp` has a `guess_func` option that can be used to supply an initial guess
value for the state variables.  This option provides the same functionality as the
`guess_nonlinear` method of [ImplicitComponent](../../../_srcdocs/packages/core/implicitcomponent).

The Kepler example script shows how `guess_func` can be used.

In [ ]:
prob = om.Problem()

bal = om.BalanceComp()

bal.add_balance(name='E', val=0.0, units='rad', eq_units='rad', rhs_name='M')

# Use M (mean anomaly) as the initial guess for E (eccentric anomaly)
def guess_function(inputs, outputs, residuals):
    if np.abs(residuals['E']) > 1.0E-2:
        outputs['E'] = inputs['M']

bal.options['guess_func'] = guess_function

# ExecComp used to compute the LHS of Kepler's equation.
lhs_comp = om.ExecComp('lhs=E - ecc * sin(E)',
                       lhs={'val': 0.0, 'units': 'rad'},
                       E={'val': 0.0, 'units': 'rad'},
                       ecc={'val': 0.0})

prob.model.add_subsystem(name='balance', subsys=bal,
                         promotes_inputs=['M'],
                         promotes_outputs=['E'])

prob.model.set_input_defaults('M', 85.0, units='deg')

prob.model.add_subsystem(name='lhs_comp', subsys=lhs_comp,
                         promotes_inputs=['E', 'ecc'])

# Explicit connections
prob.model.connect('lhs_comp.lhs', 'balance.lhs:E')

# Set up solvers
prob.model.linear_solver = om.DirectSolver()
prob.model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False, maxiter=100, iprint=2)

prob.setup()

prob.set_val('ecc', 0.6)

prob.run_model()

print(np.degrees(prob.get_val('E')))

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

assert_almost_equal(np.degrees(prob.get_val('E')), 115.9, decimal=1)